In [1]:
import torch
from sklearn.metrics import f1_score
from graph_qa_dataset import GraphQADataset
from torch_geometric.loader import DataListLoader
from model.graph_qa_model import GAT, HGT

In [2]:
train_dataset = GraphQADataset(split="train", data_size=2)
val_dataset = GraphQADataset(split="validation", data_size=1)

Found cached dataset squad (/home/mingzhe/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


  0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2022-10-31 02:25:53 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_met

2022-10-31 02:25:54 INFO: Loading these models for language: en (English):
| Processor    | Package  |
---------------------------
| tokenize     | combined |
| pos          | combined |
| constituency | wsj      |

2022-10-31 02:25:54 INFO: Use device: gpu
2022-10-31 02:25:54 INFO: Loading: tokenize
2022-10-31 02:25:56 INFO: Loading: pos
2022-10-31 02:25:56 INFO: Loading: constituency
2022-10-31 02:25:57 INFO: Done loading processors!
100%|██████████| 2/2 [00:12<00:00,  6.29s/it]
Found cached dataset squad (/home/mingzhe/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


  0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2022-10-31 02:26:32 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_met

2022-10-31 02:26:33 INFO: Loading these models for language: en (English):
| Processor    | Package  |
---------------------------
| tokenize     | combined |
| pos          | combined |
| constituency | wsj      |

2022-10-31 02:26:33 INFO: Use device: gpu
2022-10-31 02:26:33 INFO: Loading: tokenize
2022-10-31 02:26:33 INFO: Loading: pos
2022-10-31 02:26:33 INFO: Loading: constituency
2022-10-31 02:26:33 INFO: Done loading processors!
100%|██████████| 1/1 [00:06<00:00,  6.85s/it]


In [3]:
train_dataloader = DataListLoader(train_dataset, batch_size=2, shuffle=True)
val_dataloader = DataListLoader(val_dataset, batch_size=1, shuffle=False)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
model = HGT(hidden_channels=64, out_channels=2, num_heads=4, num_layers=2, metadata=train_dataset.metadata)

In [6]:
loss_op = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [7]:
model.train()

data = train_dataset.graph_data[0][0]

print(data)

HeteroData(
  question={
    x=[27, 768],
    y=[27]
  },
  context={
    x=[260, 768],
    y=[260]
  },
  (question, connect, question)={ edge_index=[2, 26] },
  (context, connect, question)={ edge_index=[2, 7] },
  (context, connect, context)={ edge_index=[2, 253] }
)


In [8]:
out = model(data.x_dict, data.edge_index_dict)

In [12]:
out

tensor([[ 0.0779, -0.0435],
        [ 0.0780, -0.0439],
        [ 0.0767, -0.0472],
        [ 0.0724, -0.0433],
        [ 0.0734, -0.0420],
        [ 0.0741, -0.0397],
        [ 0.0642, -0.0435],
        [ 0.0602, -0.0428],
        [ 0.0632, -0.0440],
        [ 0.0714, -0.0461],
        [ 0.0626, -0.0442],
        [ 0.0685, -0.0486],
        [ 0.0652, -0.0442],
        [ 0.0700, -0.0421],
        [ 0.0655, -0.0436],
        [ 0.0728, -0.0459],
        [ 0.0736, -0.0483],
        [ 0.0763, -0.0461],
        [ 0.0760, -0.0482],
        [ 0.0698, -0.0475],
        [ 0.0657, -0.0412],
        [ 0.0696, -0.0459],
        [ 0.0654, -0.0447],
        [ 0.0602, -0.0428],
        [ 0.0674, -0.0417],
        [ 0.0673, -0.0434],
        [ 0.0623, -0.0470],
        [ 0.0691, -0.0436],
        [ 0.0630, -0.0427],
        [ 0.0678, -0.0440],
        [ 0.0657, -0.0430],
        [ 0.0782, -0.0520],
        [ 0.0649, -0.0498],
        [ 0.0652, -0.0442],
        [ 0.0657, -0.0409],
        [ 0.0659, -0

In [9]:
data.x_dict

{'question': tensor([[-0.8143, -0.4067, -0.5450,  ..., -0.4847, -0.6654,  0.7649],
         [-0.8143, -0.4067, -0.5450,  ..., -0.4847, -0.6654,  0.7649],
         [-0.8143, -0.4067, -0.5450,  ..., -0.4847, -0.6654,  0.7649],
         ...,
         [-0.8603, -0.3164, -0.0064,  ...,  0.2306, -0.6456,  0.8738],
         [-0.9448, -0.4524, -0.2776,  ..., -0.2155, -0.6419,  0.9170],
         [-0.9547, -0.4047, -0.8256,  ..., -0.6352, -0.7834,  0.9697]],
        grad_fn=<StackBackward0>),
 'context': tensor([[-0.8902, -0.2453, -0.8936,  ..., -0.8152, -0.5622,  0.8019],
         [-0.8902, -0.2453, -0.8936,  ..., -0.8152, -0.5622,  0.8019],
         [-0.8902, -0.2453, -0.8936,  ..., -0.8152, -0.5622,  0.8019],
         ...,
         [-0.7895, -0.2154,  0.4098,  ...,  0.4250, -0.6165,  0.8128],
         [-0.7895, -0.2154,  0.4098,  ...,  0.4250, -0.6165,  0.8128],
         [-0.9174, -0.3771, -0.8672,  ..., -0.6802, -0.7260,  0.9148]],
        grad_fn=<StackBackward0>)}

In [11]:
data.edge_index_dict

{('question',
  'connect',
  'question'): tensor([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
          19, 20, 21, 22, 23, 24, 25, 26],
         [ 0,  1,  2,  3,  3,  5,  2,  7,  7,  9,  9,  9,  7, 13,  7, 15, 15, 17,
          17, 19, 15, 21, 21, 23, 23,  2]]),
 ('context',
  'connect',
  'question'): tensor([[0, 1, 2, 3, 4, 5, 6],
         [0, 0, 0, 0, 0, 0, 0]]),
 ('context',
  'connect',
  'context'): tensor([[  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
           15,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,
           30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  44,
           45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,
           59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,
           73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,
           87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98, 100, 101,
  